In [1]:
from datasets import load_dataset

ds = load_dataset("Helsinki-NLP/opus-100", "en-zh")

README.md: 0.00B [00:00, ?B/s]

en-zh/test-00000-of-00001.parquet:   0%|          | 0.00/355k [00:00<?, ?B/s]

en-zh/train-00000-of-00001.parquet:   0%|          | 0.00/143M [00:00<?, ?B/s]

en-zh/validation-00000-of-00001.parquet:   0%|          | 0.00/359k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [2]:
ds["train"]['translation']

Column([{'en': 'Sixty-first session', 'zh': '第六十一届会议'}, {'en': 'I took some medicine for my mu for my mu my muscular for my muscular...', 'zh': '减轻酸... 酸痛的药 减轻酸痛的药'}, {'en': "It's a challenge. God is challenging you. He's calling you a chump.", 'zh': '上帝在挑战你，他说你是笨蛋'}, {'en': 'Oh, baby.', 'zh': '.. 寶貝'}, {'en': '- Lucinda?', 'zh': '- 盧辛達？'}])

In [3]:
with open("corpus.txt", "w", encoding="utf-8") as f:
    for example in ds["train"]:    # <--- lặp dataset 'train' chứ không phải DatasetDict
        en = example["translation"]["en"]
        zh = example["translation"]["zh"]
        f.write(en + "\n")
        f.write(zh + "\n")

In [5]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer
from tokenizers.pre_tokenizers import Whitespace

tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
tokenizer.pre_tokenizer = None

trainer = WordPieceTrainer(
    vocab_size=30000,
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]
)

tokenizer.train(files=["corpus.txt"], trainer=trainer)
tokenizer.save("wordpiece.json")


In [9]:
from transformers.models.t5gemma.modeling_t5gemma import T5GemmaModuleConfig, T5GemmaModel
from transformers.models.t5gemma.configuration_t5gemma import T5GemmaConfig
# import tokenizers.
# encoder = 
encoder_config = T5GemmaModuleConfig(
    num_hidden_layers=4,
    attention_dropout=0.2,
    hidden_size=1024,
    vocab_size=30000
)
decoder_config = T5GemmaModuleConfig(
    num_hidden_layers=4,
    attention_dropout=0.2,
    hidden_size=1024,
    vocab_size=30000
)

config = T5GemmaConfig(encoder=encoder_config, decoder=decoder_config)

In [10]:
model = T5GemmaModel(config)

In [11]:
print(model)

T5GemmaModel(
  (encoder): T5GemmaEncoder(
    (embed_tokens): Embedding(30000, 1024, padding_idx=0)
    (norm): T5GemmaRMSNorm((1024,), eps=1e-06)
    (rotary_emb): T5GemmaRotaryEmbedding()
    (layers): ModuleList(
      (0-3): 4 x T5GemmaEncoderLayer(
        (self_attn): T5GemmaSelfAttention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
        )
        (pre_self_attn_layernorm): T5GemmaRMSNorm((1024,), eps=1e-06)
        (post_self_attn_layernorm): T5GemmaRMSNorm((1024,), eps=1e-06)
        (mlp): T5GemmaMLP(
          (gate_proj): Linear(in_features=1024, out_features=9216, bias=False)
          (up_proj): Linear(in_features=1024, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=102

In [15]:
from tokenizers import Tokenizer

# load tokenizer đã train
tokenizer = Tokenizer.from_file("wordpiece.json")

# cấu hình truncation/padding cho chiều dài cố định
tokenizer.enable_truncation(max_length=128)
tokenizer.enable_padding(length=128)

def tokenize_fn(batch):
    # batch['translation'] là list of dicts {'en':..., 'zh':...}
    sources = [t["en"] for t in batch["translation"]]
    targets = [t["zh"] for t in batch["translation"]]

    enc_src = tokenizer.encode_batch(sources)
    enc_tgt = tokenizer.encode_batch(targets)

    return {
        "input_ids": [enc.ids for enc in enc_src],
        "attention_mask": [enc.attention_mask for enc in enc_src],
        "labels": [enc.ids for enc in enc_tgt],
    }

# Áp tokenizer lên split train (hoặc tất cả các split)
tokenized_train = ds["train"].map(tokenize_fn, batched=True, batch_size=1000)

# Chuyển định dạng sang PyTorch tensors
tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

In [17]:
tokenized_train[0]

{'input_ids': tensor([29756, 20112, 19782, 19050, 18517,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [19]:
# ...existing code...
# Split tokenized dataset -> train/val, convert labels pad -> -100, set format và tạo DataLoader
from torch.utils.data import DataLoader

BATCH_SIZE = 32
TEST_SIZE = 0.1
SEED = 42

# xác định id token [PAD]
pad_id = tokenizer.token_to_id("[PAD]") if tokenizer.token_to_id("[PAD]") is not None else 0

def _labels_to_ignore_index(batch):
    # chuyển pad token trong labels thành -100 để loss ignore
    batch["labels"] = [[(t if t != pad_id else -100) for t in labs] for labs in batch["labels"]]
    return batch

# áp chuyển đổi cho toàn bộ tokenized dataset (nếu chưa làm)
tokenized_train = tokenized_train.map(_labels_to_ignore_index, batched=True, batch_size=1000)

# chia train/val
splits = tokenized_train.train_test_split(test_size=TEST_SIZE, seed=SEED)
train_ds = splits["train"]
val_ds = splits["test"]

# set format sang torch tensors
cols = ["input_ids", "attention_mask", "labels"]
train_ds.set_format(type="torch", columns=cols)
val_ds.set_format(type="torch", columns=cols)

# DataLoader (mặc định collate tốt vì đã padding)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

print("Train samples:", len(train_ds), "Val samples:", len(val_ds))
# ...existing code...

Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Train samples: 900000 Val samples: 100000
